## Importing the Necessary Libraries

In [43]:
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, Flatten, MaxPooling1D

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Setting the Hyperparameters
These will be required for building the neural network. We can play around with these and they will largely affect the accuracy of our model

In [44]:
vocab_size = 1000
max_len =1000
batch_size = 32
embedding_dims =10
filters = 16
ker_size = 3 # kernel size
hidden_dims = 250
epochs = 10

## Reading the Dataset from CSV file

In [45]:
data = pd.read_csv('mbti_cleaned.csv')
data.dropna(inplace=True)  # ignoring the NaN values

In [124]:
# One Hot encoding on the dataset output classes 
y = pd.DataFrame(data[['Is E','Is S','Is T', 'Is J']])
x_train,x_test,y_train,y_test = train_test_split(data['Posts'], y,random_state=0)
y

In [47]:
tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)

In [48]:
x_train = tokenizer.texts_to_matrix(x_train)
x_test = tokenizer.texts_to_matrix(x_test)

In [49]:
x_train = sequence.pad_sequences(x_train,maxlen=max_len)
x_test = sequence.pad_sequences(x_test,maxlen=max_len)

## Building the Sequential Neural Network using Keras

In [50]:
model = Sequential()
# First we add an embedding layer 
model.add(Embedding(vocab_size,embedding_dims,input_length=max_len)) 
# Adding a 1D convolutional Layer
model.add(Conv1D(filters, ker_size, padding='valid', activation='relu'))
# Max Pooling the Convolutions
model.add(MaxPooling1D())
# Again Computing the Convolutions
model.add(Flatten())
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dense(4, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [53]:
# Training the modeL
model.fit(x_train,y_train, batch_size=batch_size, epochs=5, validation_data=(x_test, y_test),)

Train on 6471 samples, validate on 2157 samples
Epoch 1/5
6471/6471 [==============================] - 12s 2ms/step - loss: 0.5496 - acc: 0.7228 - val_loss: 0.4991 - val_acc: 0.7594
Epoch 2/5
6471/6471 [==============================] - 11s 2ms/step - loss: 0.4533 - acc: 0.7854 - val_loss: 0.4756 - val_acc: 0.7729
Epoch 3/5
6471/6471 [==============================] - 12s 2ms/step - loss: 0.4022 - acc: 0.8147 - val_loss: 0.4713 - val_acc: 0.7774
Epoch 4/5
6471/6471 [==============================] - 12s 2ms/step - loss: 0.3650 - acc: 0.8334 - val_loss: 0.4924 - val_acc: 0.7714
Epoch 5/5
6471/6471 [==============================] - 12s 2ms/step - loss: 0.3288 - acc: 0.8552 - val_loss: 0.5321 - val_acc: 0.7694


## Evaluating the Performance of the Model

In [54]:
model.evaluate(x_test,y_test)[1]*100  # Accuracy of the model

2157/2157 [==============================] - 0s 199us/step


76.93555865732121

In [55]:
model.evaluate(x_test,y_test)[1]*100  # Accuracy of the model

2157/2157 [==============================] - 0s 205us/step


76.93555865732121

In [39]:
# Saving the model into a pickle file 
import pickle
pickle.dump(model,open('cnn_model.pkl','wb'))

In [40]:
pickle.dump(tokenizer,open('tokenizer','wb'))

# For Single Input 

In [119]:
s ='Idealistic, loyal to their values and to people who are important to them. Want an external life that is congruent with their values. Curious, quick to see possibilities, can be catalysts for implementing ideas. Seek to understand people and to help them fulfill their potential. Adaptable, flexible, and accepting unless a value is threatened.'
s = pd.Series(s)
s= tokenizer.texts_to_matrix(s)
s = sequence.pad_sequences(s)
l = model.predict(s)

In [120]:
a,b= l[0][0]*(1/1999), l[0][1]*(1/1197)
a = a/(1/1999)+(1/1197)
b = b/(1/1999)+(1/1197)

In [121]:
l = [a,b,l[0][2],l[0][3]]

In [126]:
s=''
if l[0] >0.5:
    s +='E'
else:
    s+='I'
if l[1] >0.5:
    s+='S'
else:
    s+='N'
if l[2] >0.5:
    s+='T'
else:
    s+='F'
if l[3] >0.5:
    s+='J'
else:
    s+='P'
print('Your Personality is:',s)

Your Personality is: INTP
